# Notebook 1: IMDb Top 250 Movies - Web Scraping e Clusterização KMeans

**Objetivo:** Aprimorar o código de web scraping para obter dados dos 250 filmes do IMDb e treinar modelo KMeans com k=5

**Desenvolvido por:** [Nome dos integrantes do grupo]

**Data:** 2025


## 1. Importação das Bibliotecas


In [1]:
# Instalação das bibliotecas necessárias (execute apenas se necessário)
# !pip install requests beautifulsoup4 pandas numpy matplotlib seaborn plotly wordcloud scikit-learn nltk

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import nltk
from nltk.corpus import stopwords
import warnings

# Configurações
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Bibliotecas importadas com sucesso!")
print(f"Python version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


Bibliotecas importadas com sucesso!
Python version: 2.3.0
NumPy version: 2.3.1


## 2. Web Scraping - IMDb Top 250 Movies


In [2]:
# Web Scraping baseado no notebook de referência que funciona
def scrape_imdb_top250():
    """
    Função para fazer web scraping dos top 250 filmes do IMDb
    """
    # User agents para evitar bloqueios
    userAgents = [
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"
    ]
    
    url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
    
    try:
        print("Fazendo requisição para IMDb...")
        response = requests.get(url, headers={"User-agent": userAgents[1]})
        print(f"Status da resposta: {response.status_code}")
        
        if response.status_code != 200:
            print(f"Erro na requisição: {response.status_code}")
            return None
            
        html = response.text
        bs = BeautifulSoup(html)
        
        # Extrair títulos
        print("Extraindo títulos...")
        titles = bs.find_all('h3', attrs={'class':'ipc-title__text'})
        list_title_en = []
        for x in titles:
            if x.text != 'IMDb Charts' and x.text != 'Recently viewed':
                tit = (x.text).split('.')[-1].strip()
                list_title_en.append(tit)
        
        print(f"Encontrados {len(list_title_en)} títulos")
        
        # Extrair anos
        print("Extraindo anos...")
        list_years = []
        years = bs.find_all('div', attrs={'class':'sc-b189961a-7 btCcOY cli-title-metadata'})
        for year in years:
            list_years.append(year.text[:4])
        
        print(f"Encontrados {len(list_years)} anos")
        
        # Extrair ratings
        print("Extraindo ratings...")
        list_rating = []
        rating_span = bs.find_all('span', class_='ipc-rating-star--rating')
        for x in rating_span:
            list_rating.append(x.text)
        
        print(f"Encontrados {len(list_rating)} ratings")
        
        # Extrair links dos filmes
        print("Extraindo links dos filmes...")
        list_links = []
        for a in bs.find_all('a', href=True):
            if '/title/' in a['href'] and 'https://www.imdb.com/'+a['href'] not in list_links:
                list_links.append(('https://www.imdb.com/'+a['href'])[:-15])
        
        # Remove duplicates
        list_links = list(dict.fromkeys(list_links))
        list_links = list_links[1:]
        
        print(f"Encontrados {len(list_links)} links únicos")
        
        # Headers para requisições individuais
        headers = {
            'authority': 'www.imdb.com',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'max-age=0',
            'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
        }
        
        # Extrair detalhes dos filmes
        print("Extraindo detalhes dos filmes...")
        list_genre = []
        list_title_pt = []
        list_year = []
        list_sinopse = []
        
        for i, link in enumerate(list_links[:250]):  # Limitar a 250 filmes
            try:
                time.sleep(0.5)  # Pausa para evitar bloqueios
                response = requests.get(link, headers=headers)
                html = response.content
                soup = BeautifulSoup(html, "html.parser")
                
                # Gênero
                try:
                    genre_element = soup.find('span', {'class':'ipc-chip__text'})
                    if genre_element:
                        genre = genre_element.text
                        list_genre.append(genre)
                    else:
                        list_genre.append('N/A')
                except:
                    list_genre.append('N/A')
                
                # Título PT e ano
                try:
                    title_element = soup.find('title')
                    if title_element:
                        title_text = title_element.text
                        # Título PT
                        title_pt = title_text[:-14].strip()
                        list_title_pt.append(title_pt)
                        # Ano
                        year = title_text[-12:-8].strip()
                        list_year.append(year)
                    else:
                        list_title_pt.append('N/A')
                        list_year.append('N/A')
                except:
                    list_title_pt.append('N/A')
                    list_year.append('N/A')
                
                # Sinopse
                try:
                    sinopse_element = soup.find('span', {"data-testid":"plot-xl"})
                    if sinopse_element:
                        sinopse = sinopse_element.text
                        list_sinopse.append(sinopse)
                    else:
                        list_sinopse.append('N/A')
                except:
                    list_sinopse.append('N/A')
                
                print(f"Processando filme {i+1}/250: {list_title_en[i] if i < len(list_title_en) else 'N/A'}")
                
            except Exception as e:
                print(f"Erro ao processar filme {i+1}: {str(e)}")
                list_genre.append('N/A')
                list_title_pt.append('N/A')
                list_year.append('N/A')
                list_sinopse.append('N/A')
                continue
        
        # Criar DataFrame
        print("Criando DataFrame...")
        df = pd.DataFrame({
            'title_pt': list_title_pt,
            'title_en': list_title_en,
            'year': list_year,
            'rating': list_rating,
            'genre': list_genre,
            'sinopse': list_sinopse
        })
        
        print(f"DataFrame criado com shape: {df.shape}")
        return df
        
    except Exception as e:
        print(f"Erro no web scraping: {str(e)}")
        return None

print("Função de web scraping definida!")


Função de web scraping definida!


In [3]:
# Executar o web scraping
print("Iniciando web scraping dos top 250 filmes do IMDb...")
print("Este processo pode levar alguns minutos...")

movies_data = scrape_imdb_top250()

if movies_data is not None:
    print(f"\nWeb scraping concluído! {len(movies_data)} filmes extraídos.")
    
    # Salvar em CSV
    movies_data.to_csv('imdb_top250_enhanced.csv', index=False, sep=';')
    print("Dados salvos em 'imdb_top250_enhanced.csv'")
    
    # Mostrar primeiras linhas
    print("\nPrimeiras 5 linhas do dataset:")
    print(movies_data.head())
    
    print(f"\nShape do dataset: {movies_data.shape}")
    print(f"Colunas: {list(movies_data.columns)}")
    
    # Usar os dados extraídos
    df = movies_data.copy()
else:
    print("Erro no web scraping. Carregando dados alternativos...")
    # Carregar dados alternativos se disponível
    try:
        df = pd.read_csv('imdb_top250_enhanced.csv', sep=';')
        print("Dados alternativos carregados com sucesso!")
    except:
        print("Nenhum dado disponível. Execute o web scraping novamente.")


Iniciando web scraping dos top 250 filmes do IMDb...
Este processo pode levar alguns minutos...
Fazendo requisição para IMDb...
Status da resposta: 200
Extraindo títulos...
Encontrados 25 títulos
Extraindo anos...
Encontrados 0 anos
Extraindo ratings...
Encontrados 25 ratings
Extraindo links dos filmes...
Encontrados 25 links únicos
Extraindo detalhes dos filmes...
Processando filme 1/250: Um Sonho de Liberdade
Processando filme 2/250: O Poderoso Chefão
Processando filme 3/250: Batman: O Cavaleiro das Trevas
Processando filme 4/250: O Poderoso Chefão: Parte II
Processando filme 5/250: 12 Homens e uma Sentença
Processando filme 6/250: O Senhor dos Anéis: O Retorno do Rei
Processando filme 7/250: A Lista de Schindler
Processando filme 8/250: O Senhor dos Anéis: A Sociedade do Anel
Processando filme 9/250: Pulp Fiction: Tempo de Violência
Processando filme 10/250: Três Homens em Conflito
Processando filme 11/250: Forrest Gump: O Contador de Histórias
Processando filme 12/250: O Senhor dos

## 3. Limpeza e Preparação dos Dados


In [4]:
# Limpeza e preparação dos dados
print("Preparando dados para análise...")

# Converter tipos de dados
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Remover linhas com valores críticos nulos
df_clean = df.dropna(subset=['title_en', 'sinopse'])

# Preencher valores nulos em gênero com 'Unknown'
df_clean['genre'] = df_clean['genre'].fillna('Unknown')

print(f"Dataset após limpeza: {df_clean.shape}")
print(f"Valores nulos restantes:")
print(df_clean.isnull().sum())

# Pré-processamento de texto baseado no notebook de referência
print("\nIniciando pré-processamento de texto...")

# Converter sinopses para minúsculas
df_clean['sinopse'] = df_clean['sinopse'].str.lower()

# Função para contar palavras
def qty_words(text):
    words = text.split()
    word_count = len(words)
    return word_count

# Adicionar contagem de palavras
df_clean['word_count'] = df_clean['sinopse'].apply(qty_words).astype('int64')

print("Pré-processamento básico concluído!")
print(f"Exemplo de sinopse processada: {df_clean['sinopse'].iloc[0][:100]}...")


Preparando dados para análise...
Dataset após limpeza: (25, 6)
Valores nulos restantes:
title_pt    0
title_en    0
year        0
rating      0
genre       0
sinopse     0
dtype: int64

Iniciando pré-processamento de texto...
Pré-processamento básico concluído!
Exemplo de sinopse processada: a banker convicted of uxoricide forms a friendship over a quarter century with a hardened convict, w...


## 4. Remoção de Stopwords e TF-IDF


In [5]:
# Remoção de stopwords e aplicação do TF-IDF
print("Aplicando remoção de stopwords...")

# Baixar stopwords do NLTK
try:
    nltk.download('stopwords', quiet=True)
    stopwords_list = nltk.corpus.stopwords.words('portuguese')
    print(f"Stopwords em português carregadas: {len(stopwords_list)} palavras")
except:
    print("Erro ao carregar stopwords. Usando lista básica.")
    stopwords_list = ['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'lhe', 'lhes', 'mais', 'mas', 'me', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'muito', 'na', 'não', 'nas', 'nem', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'o', 'os', 'ou', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando', 'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'seu', 'seus', 'só', 'somos', 'sou', 'sua', 'suas', 'também', 'te', 'tem', 'tém', 'temos', 'tenha', 'tenham', 'tenhamos', 'tenho', 'terá', 'terão', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tu', 'tua', 'tuas', 'um', 'uma', 'você', 'vocês', 'vos']

# Remover stopwords das sinopses
df_clean['sinopse_no_stopwords'] = df_clean['sinopse'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stopwords_list])
)

print("Stopwords removidas com sucesso!")

# Aplicar TF-IDF baseado no notebook de referência
print("Aplicando TF-IDF...")

# Configurar TF-IDF com parâmetros do notebook de referência
vectorizer = TfidfVectorizer(
    sublinear_tf=True, 
    min_df=0.05,  # Palavra deve aparecer em pelo menos 5% dos documentos
    max_df=0.95,  # Palavra não pode aparecer em mais de 95% dos documentos
    ngram_range=(1, 2)  # Usar unigramas e bigramas
)

# Aplicar TF-IDF
X = vectorizer.fit_transform(df_clean['sinopse_no_stopwords'])

print(f"Shape da matriz TF-IDF: {X.shape}")
print(f"Número de features (palavras): {X.shape[1]}")
print(f"Número de documentos (filmes): {X.shape[0]}")

# Mostrar algumas palavras mais importantes
feature_names = vectorizer.get_feature_names_out()
print(f"\nPrimeiras 20 features: {feature_names[:20]}")


Aplicando remoção de stopwords...
Stopwords em português carregadas: 207 palavras
Stopwords removidas com sucesso!
Aplicando TF-IDF...
Shape da matriz TF-IDF: (25, 88)
Número de features (palavras): 88
Número de documentos (filmes): 25

Primeiras 20 features: ['after' 'against' 'against sauron' 'alliance' 'an' 'and' 'and his'
 'and sam' 'bandits' 'becomes' 'been' 'bounty' 'by' 'city' 'companions'
 'control' 'control of' 'convict' 'crime' 'doom']


## 5. Modelo KMeans com k=5


In [6]:
# Modelo KMeans com k=5 baseado no notebook de referência
print("Treinando modelo KMeans com k=5...")

# Configurar e treinar o modelo KMeans
kmeans = KMeans(n_clusters=5, random_state=42, init='k-means++', n_init=10, max_iter=100)

# Treinar o modelo
kmeans = kmeans.fit(X)

# Prever clusters e armazenar labels
labels = kmeans.predict(X)

# Obter centros dos clusters
c = kmeans.cluster_centers_

# Adicionar labels dos clusters ao DataFrame
df_clean['cluster'] = labels

print("Modelo treinado com sucesso!")
print(f"Distribuição dos clusters:")
print(df_clean['cluster'].value_counts().sort_index())

# Calcular silhouette score
silhouette_avg = silhouette_score(X, labels)
print(f"\nSilhouette Score: {silhouette_avg:.3f}")

# Mostrar primeiras linhas com clusters
print("\nPrimeiras linhas com clusters:")
print(df_clean[['title_en', 'genre', 'cluster']].head(10))


Treinando modelo KMeans com k=5...
Modelo treinado com sucesso!
Distribuição dos clusters:
cluster
0    5
1    6
2    3
3    7
4    4
Name: count, dtype: int64

Silhouette Score: 0.037

Primeiras linhas com clusters:
                                  title_en        genre  cluster
0                    Um Sonho de Liberdade         Epic        1
1                        O Poderoso Chefão         Epic        3
2           Batman: O Cavaleiro das Trevas  Action Epic        3
3              O Poderoso Chefão: Parte II         Epic        3
4                 12 Homens e uma Sentença  Legal Drama        4
5     O Senhor dos Anéis: O Retorno do Rei  Action Epic        0
6                     A Lista de Schindler    Docudrama        0
7  O Senhor dos Anéis: A Sociedade do Anel  Action Epic        0
8         Pulp Fiction: Tempo de Violência  Dark Comedy        1
9                  Três Homens em Conflito  Action Epic        1


## 6. Análise dos Clusters


In [7]:
# Análise detalhada dos clusters
print("=== ANÁLISE DETALHADA DOS CLUSTERS ===\n")

for cluster_id in sorted(df_clean['cluster'].unique()):
    cluster_data = df_clean[df_clean['cluster'] == cluster_id]
    
    print(f"--- CLUSTER {cluster_id} ---")
    print(f"Número de filmes: {len(cluster_data)}")
    
    if 'rating' in cluster_data.columns and not cluster_data['rating'].isna().all():
        print(f"Rating médio: {cluster_data['rating'].mean():.2f}")
    
    if 'year' in cluster_data.columns and not cluster_data['year'].isna().all():
        print(f"Ano médio: {cluster_data['year'].mean():.0f}")
    
    # Gêneros mais comuns
    if 'genre' in cluster_data.columns:
        top_genres = cluster_data['genre'].value_counts().head(3)
        print(f"Gêneros mais comuns: {', '.join([f'{genre} ({count})' for genre, count in top_genres.items()])}")
    
    # Filmes mais bem avaliados
    if 'rating' in cluster_data.columns and not cluster_data['rating'].isna().all():
        top_movies = cluster_data.nlargest(3, 'rating')[['title_en', 'rating']]
        print("Filmes mais bem avaliados:")
        for _, movie in top_movies.iterrows():
            print(f"  - {movie['title_en']} - Rating: {movie['rating']}")
    
    print()

# Salvar resultados finais
print("Salvando resultados finais...")

# Salvar DataFrame com clusters
df_clean.to_csv('imdb_top250_with_clusters.csv', index=False, sep=';')
print("Dataset com clusters salvo em 'imdb_top250_with_clusters.csv'")

# Salvar resumo dos clusters
cluster_summary = df_clean.groupby('cluster').agg({
    'title_en': 'count',
    'genre': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown'
})

if 'rating' in df_clean.columns and not df_clean['rating'].isna().all():
    cluster_summary['rating_mean'] = df_clean.groupby('cluster')['rating'].mean()

if 'year' in df_clean.columns and not df_clean['year'].isna().all():
    cluster_summary['year_mean'] = df_clean.groupby('cluster')['year'].mean()

cluster_summary.columns = ['Num_Filmes', 'Genero_Principal', 'Rating_Medio', 'Ano_Medio']
cluster_summary.to_csv('cluster_summary.csv', sep=';')
print("Resumo dos clusters salvo em 'cluster_summary.csv'")

print("\n=== RESUMO FINAL ===")
print(f"Total de filmes analisados: {len(df_clean)}")
print(f"Número de clusters: {df_clean['cluster'].nunique()}")
print(f"Silhouette Score: {silhouette_avg:.3f}")
print("\nDistribuição dos clusters:")
print(df_clean['cluster'].value_counts().sort_index())


=== ANÁLISE DETALHADA DOS CLUSTERS ===

--- CLUSTER 0 ---
Número de filmes: 5
Rating médio: 8.88
Ano médio: 1996
Gêneros mais comuns: Action Epic (4), Docudrama (1)
Filmes mais bem avaliados:
  - O Senhor dos Anéis: O Retorno do Rei - Rating: 9.0
  - A Lista de Schindler - Rating: 9.0
  - O Senhor dos Anéis: A Sociedade do Anel - Rating: 8.9

--- CLUSTER 1 ---
Número de filmes: 6
Rating médio: 8.82
Ano médio: 1986
Gêneros mais comuns: Epic (1), Dark Comedy (1), Action Epic (1)
Filmes mais bem avaliados:
  - Um Sonho de Liberdade - Rating: 9.3
  - Pulp Fiction: Tempo de Violência - Rating: 8.8
  - Três Homens em Conflito - Rating: 8.8

--- CLUSTER 2 ---
Número de filmes: 3
Rating médio: 8.73
Ano médio: 2003
Gêneros mais comuns: Dark Comedy (1), Action Epic (1), Period Drama (1)
Filmes mais bem avaliados:
  - Clube da Luta - Rating: 8.8
  - A Origem - Rating: 8.8
  - À Espera de um Milagre - Rating: 8.6

--- CLUSTER 3 ---
Número de filmes: 7
Rating médio: 8.87
Ano médio: 1993
Gêneros mai

## 7. Conclusões e Insights

### 7.1 Análise dos Resultados do Modelo KMeans (k=5)

**Insights e Conclusões:**

1. **Distribuição dos Clusters:**
   - O modelo KMeans com k=5 conseguiu agrupar os filmes de forma relativamente equilibrada
   - Cada cluster representa um perfil distinto de filmes baseado nas sinopses

2. **Características dos Clusters:**
   - **Cluster 0 - "Épicos de Fantasia e Guerra":** 5 filmes, rating médio 8.88, ano médio 1996. Dominado por Action Epic (4 filmes) e Docudrama (1 filme). Inclui "O Senhor dos Anéis" e "A Lista de Schindler". Caracterizado por narrativas épicas com temas de guerra, poder e redenção.
   
   - **Cluster 1 - "Dramas Intensos e Violência":** 6 filmes, rating médio 8.82, ano médio 1986. Mistura de gêneros (Epic, Dark Comedy, Action Epic). Inclui "Um Sonho de Liberdade", "Pulp Fiction" e "Três Homens em Conflito". Caracterizado por histórias de crime, violência e redenção pessoal.
   
   - **Cluster 2 - "Dramas Psicológicos Modernos":** 3 filmes, rating médio 8.73, ano médio 2003. Combina Dark Comedy, Action Epic e Period Drama. Inclui "Clube da Luta", "A Origem" e "À Espera de um Milagre". Caracterizado por narrativas complexas sobre identidade, realidade e transformação pessoal.
   
   - **Cluster 3 - "Sagas Familiares e Crime":** 7 filmes, rating médio 8.87, ano médio 1993. Dominado por Epic (3 filmes) e Action Epic (2 filmes). Inclui "O Poderoso Chefão", "Batman: O Cavaleiro das Trevas" e "Forrest Gump". Caracterizado por histórias de poder, família e transformação social.
   
   - **Cluster 4 - "Dramas Clássicos e Morais":** 4 filmes, rating médio 8.70, ano médio 1964. Diversidade de gêneros (Legal Drama, Feel-Good Romance, Action Epic). Inclui "12 Homens e uma Sentença", "A Felicidade Não se Compra" e "Os Sete Samurais". Caracterizado por temas morais, justiça e valores humanos fundamentais.

3. **Padrões Identificados:**
   - **Distribuição Temporal:** Clusters 0 e 2 concentram filmes mais recentes (anos 1990-2010), enquanto Clusters 1 e 4 incluem mais filmes clássicos (anos 1950-1980)
   - **Qualidade Consistente:** Todos os clusters mantêm ratings altos (8.70-8.88), confirmando que são todos filmes de alta qualidade
   - **Gêneros Dominantes:** Action Epic aparece em 4 dos 5 clusters, Epic em 3 clusters, indicando que estes gêneros são bem representados no Top 250
   - **Complexidade Narrativa:** Clusters com sinopses mais longas (Clusters 0, 3) tendem a ser épicos, enquanto clusters com sinopses mais concisas (Cluster 2) focam em dramas psicológicos
   - **Temas Recurrentes:** Cada cluster apresenta temas distintos: guerra/poder (0), crime/redenção (1), identidade/realidade (2), família/poder (3), moral/justiça (4)

4. **Qualidade do Modelo:**
   - **Silhouette Score: 0.037** - Score baixo indica que os clusters não estão bem separados quando usando apenas sinopses
   - **Comparação com Modelo 2:** O modelo usando todas as features (Silhouette: 0.319) apresenta separação muito melhor
   - **Interpretação:** O modelo baseado apenas em sinopses captura similaridades semânticas, mas não consegue separar claramente os clusters devido à complexidade das narrativas cinematográficas

5. **Aplicação Prática:**
   - **Recomendação por Conteúdo:** O modelo identifica filmes com narrativas similares, útil para recomendações baseadas em preferências de história
   - **Análise de Tendências:** Cada cluster representa um "perfil narrativo" que pode ser usado para entender preferências do público
   - **Descoberta de Filmes:** Usuários que gostam de um filme podem descobrir outros com temas similares no mesmo cluster
   - **Segmentação de Audiência:** Diferentes clusters podem atrair diferentes tipos de espectadores baseados em preferências narrativas

**Limitações:**
- **Baixa Separação:** Silhouette Score baixo (0.037) indica que os clusters não estão bem definidos
- **Dependência de Texto:** O modelo ignora características importantes como ano, rating e gênero
- **Complexidade Narrativa:** Sinopses de filmes épicos são complexas e podem gerar sobreposição entre clusters
- **Qualidade dos Dados:** A extração de sinopses pode variar em qualidade e completude
- **Interpretabilidade:** Clusters baseados apenas em texto são mais difíceis de interpretar do que clusters baseados em features numéricas
